# PSP analysis 2024.06.23

In [47]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.metrics import  accuracy_score
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier, Pool, metrics, cv
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, accuracy_score, auc
from sklearn.metrics import roc_auc_score, confusion_matrix, precision_recall_curve

from sklearn.preprocessing import StandardScaler

from xgboost import XGBClassifier

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedKFold

In [3]:
path1=Path("/Users/zainabnazari/Desktop/psp")

In [4]:
psp_data=pd.read_csv(path1/"input_RS_PSPP_18_predictors.txt",delimiter='\t')
psp_data

,ID,Diagnosis_subtype_RS0_PSPP1,ENST00000238156,ENST00000257262,ENST00000284503,ENST00000351111,ENST00000374893,ENST00000380189,ENST00000394095,ENST00000419654,ENST00000451310,ENST00000498316,ENST00000510702,ENST00000511320,ENST00000539642,ENST00000544491,ENST00000560567,ENST00000600480,ENST00000646269,ENST00000651723
0,PSP001,0,3.002266,0.178795,2.033664,3.719518,6.150829,-0.478296,-1.444489,0.874005,-0.760221,0.095220,-3.630769,3.489275,1.816005,-0.415653,-0.543782,2.202923,-0.684409,2.308744
1,PSP002,1,0.249801,0.889648,1.132954,3.644508,7.295879,-3.923761,3.757737,-2.307350,-0.454877,1.597854,-3.923761,-0.488927,1.751582,-1.128809,0.779541,1.177052,3.339594,-1.717212
2,PSP003,1,0.982581,0.744061,1.464088,0.302467,7.009042,0.673183,-1.285566,1.584857,-0.759069,0.844248,0.499700,-0.300993,1.888452,-1.357953,0.520039,0.325762,5.360383,-1.690411
3,PSP004,1,0.691634,-0.224783,0.511230,1.722467,6.936919,-1.690061,-0.224783,-0.586673,-0.002731,1.261970,1.812942,-1.070876,2.455030,-1.304586,0.604250,0.907776,4.287762,-0.109489
4,PSP005,1,0.920849,2.029771,1.290740,0.208501,7.248590,-1.259305,-1.259305,-2.606945,0.119454,0.345691,0.345691,-0.077045,2.627923,-2.606945,-1.038277,-0.971536,0.056882,-3.797290
5,PSP006,1,1.271013,0.285434,1.314521,0.969132,6.649289,-1.953298,0.754236,-3.349114,-0.700513,0.115452,-0.700513,-0.214828,1.754401,-2.176049,0.225914,-0.729677,3.622272,-0.952223
6,PSP007,0,2.426148,0.036409,2.030196,2.416220,4.714437,-0.972636,-1.465340,-2.348882,-3.263152,0.708127,0.207335,2.349972,-0.871891,-0.777725,-4.354783,1.332407,-0.281299,0.535457
7,PSP008,1,0.782280,0.364415,0.550198,-0.329346,7.362393,-1.715795,0.099115,-0.836513,0.017410,1.904840,0.782280,-0.941536,3.047212,-1.243367,0.714762,-0.738620,3.913413,0.319668
8,PSP009,1,2.399703,1.562357,1.879502,-3.041134,7.101801,-1.045292,3.442025,-4.354783,1.201540,0.091455,-4.354783,-1.045292,2.912252,-0.667100,-4.354783,-1.559270,1.760873,-1.559270
9,PSP010,0,1.710236,-0.015965,1.045036,3.493541,5.348468,2.027848,-1.378451,1.451505,-3.107484,-1.378451,-4.354783,1.739544,0.843833,-1.378451,-2.448689,1.824055,0.078810,3.632820


In [12]:
# Separate features (X) and target variable (y)
X1 = psp_data.drop(['ID', 'Diagnosis_subtype_RS0_PSPP1'], axis=1)

y = psp_data['Diagnosis_subtype_RS0_PSPP1']

In [45]:
X1
X = X1.sample(n=18, axis=1)
X

,ENST00000394095,ENST00000646269,ENST00000600480,ENST00000539642,ENST00000511320,ENST00000419654,ENST00000238156,ENST00000351111,ENST00000560567,ENST00000651723,ENST00000510702,ENST00000451310,ENST00000498316,ENST00000257262,ENST00000380189,ENST00000284503,ENST00000544491,ENST00000374893
0,-1.444489,-0.684409,2.202923,1.816005,3.489275,0.874005,3.002266,3.719518,-0.543782,2.308744,-3.630769,-0.760221,0.095220,0.178795,-0.478296,2.033664,-0.415653,6.150829
1,3.757737,3.339594,1.177052,1.751582,-0.488927,-2.307350,0.249801,3.644508,0.779541,-1.717212,-3.923761,-0.454877,1.597854,0.889648,-3.923761,1.132954,-1.128809,7.295879
2,-1.285566,5.360383,0.325762,1.888452,-0.300993,1.584857,0.982581,0.302467,0.520039,-1.690411,0.499700,-0.759069,0.844248,0.744061,0.673183,1.464088,-1.357953,7.009042
3,-0.224783,4.287762,0.907776,2.455030,-1.070876,-0.586673,0.691634,1.722467,0.604250,-0.109489,1.812942,-0.002731,1.261970,-0.224783,-1.690061,0.511230,-1.304586,6.936919
4,-1.259305,0.056882,-0.971536,2.627923,-0.077045,-2.606945,0.920849,0.208501,-1.038277,-3.797290,0.345691,0.119454,0.345691,2.029771,-1.259305,1.290740,-2.606945,7.248590
5,0.754236,3.622272,-0.729677,1.754401,-0.214828,-3.349114,1.271013,0.969132,0.225914,-0.952223,-0.700513,-0.700513,0.115452,0.285434,-1.953298,1.314521,-2.176049,6.649289
6,-1.465340,-0.281299,1.332407,-0.871891,2.349972,-2.348882,2.426148,2.416220,-4.354783,0.535457,0.207335,-3.263152,0.708127,0.036409,-0.972636,2.030196,-0.777725,4.714437
7,0.099115,3.913413,-0.738620,3.047212,-0.941536,-0.836513,0.782280,-0.329346,0.714762,0.319668,0.782280,0.017410,1.904840,0.364415,-1.715795,0.550198,-1.243367,7.362393
8,3.442025,1.760873,-1.559270,2.912252,-1.045292,-4.354783,2.399703,-3.041134,-4.354783,-1.559270,-4.354783,1.201540,0.091455,1.562357,-1.045292,1.879502,-0.667100,7.101801
9,-1.378451,0.078810,1.824055,0.843833,1.739544,1.451505,1.710236,3.493541,-2.448689,3.632820,-4.354783,-3.107484,-1.378451,-0.015965,2.027848,1.045036,-1.378451,5.348468


In [46]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# Assuming X and y are your DataFrame and Series
# X = pd.DataFrame( ... )
# y = pd.Series( ... )

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Initialize and train the XGBoost model
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_prob = model.predict_proba(X_test)[:, 1]
y_pred = model.predict(X_test)

# Compute the AUC score
auc = roc_auc_score(y_test, y_pred_prob)
print(f'AUC Score: {auc}')

# Create a DataFrame with the predicted and real labels
results = pd.DataFrame({'Real Label': y_test, 'Predicted Probability': y_pred_prob, 'Predicted Label': y_pred})
print(results)

# Print the DataFrame
print("Predicted values along with real labels:")
print(results.to_string(index=False))


AUC Score: 1.0
    Real Label  Predicted Probability  Predicted Label
8            1               0.878507                1
16           0               0.507770                1
0            0               0.476646                0
18           0               0.399106                0
11           1               0.703520                1
9            0               0.311421                0
13           0               0.669309                1
1            1               0.753130                1
21           1               0.821952                1
5            1               0.826943                1
2            1               0.676897                1
12           0               0.399106                0
Predicted values along with real labels:
 Real Label  Predicted Probability  Predicted Label
          1               0.878507                1
          0               0.507770                1
          0               0.476646                0
          0          

In [1]:
# Generate some random data
X = np.array([[1], [2], [3], [4], [5]])
y = np.array([2, 3, 5, 7, 11])

# Create a linear regression model
model = LinearRegression()

# Fit the model to the data
model.fit(X, y)

# Print the coefficients
print("Coefficient: ", model.coef_)

# Print the intercept
print("Intercept: ", model.intercept_)

# Make predictions
y_pred = model.predict(X)

# Print the predicted values
print("Predicted values: ", y_pred)


Coefficient:  [2.2]
Intercept:  -1.0000000000000018
Predicted values:  [ 1.2  3.4  5.6  7.8 10. ]
